In [ ]:
!pip install torch==1.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Dataloader

In [ ]:
import pandas as pd
import numpy as np
import os

import torch
from torch.utils.data import DataLoader, Dataset, IterableDataset
from itertools import count, islice

import time
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from tqdm import tqdm

### Доступ к данным

In [ ]:
filename = "outoutGeneralRelativityAndQuantumCosmology_0_result.feather"
df = pd.read_feather(filename)
title_list = df["Title"].tolist()
annotation_list = df["Annotation"].tolist()
clear_list = df["clear"].tolist()

## Preparing Data

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# import torchtext
# from torchtext.legacy.data import Field, BucketIterator

import spacy

import random
import math
import time
import numpy as np

import matplotlib
matplotlib.rcParams.update({'figure.figsize': (16, 12), 'font.size': 14})
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

from nltk.tokenize import WordPunctTokenizer

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
tokenizer_W = WordPunctTokenizer()

def tokenize_ru_in(x, tokenizer=tokenizer_W):
    return tokenizer.tokenize(x.lower())[::-1]

def tokenize_ru_out(x, tokenizer=tokenizer_W):
    return tokenizer.tokenize(x.lower())

In [ ]:
def collect_data(df):
    i = 0
    for title in df["Title"].tolist():
        annotation = df[df["Title"] == title]['Annotation'].tolist()[0]
        clear = df[df["Title"] == title]['clear'].tolist()[0]
        if not(clear is None or annotation is None):
          data_dict[i] = (title, tokenize_ru_out(annotation), tokenize_ru_in(clear))
          i += 1
    return data_dict

In [ ]:
data_dict = {}
data_dict = collect_data(df)
data = list(data_dict.keys())

In [ ]:
len(data_dict)

1021

In [ ]:
# def collect_data_list(df):
#     data_list = []
#     for title in df["Title"].tolist():
#         annotation = df[df["Title"] == title]['Annotation'].tolist()[0]
#         clear = df[df["Title"] == title]['clear'].tolist()[0]
#         if not(clear is None or annotation is None):
#           data_list.append({'title': title, 'annotation': tokenize_ru_out(annotation), 'clear': tokenize_ru_in(clear)})
#     return data_list

In [ ]:
# data_list = []
# data_list = collect_data_list(df)
# len(data_list)

1021

In [ ]:
from torch.utils.data import DataLoader, Dataset, IterableDataset
import time

In [ ]:
class MyIterableDataset(IterableDataset):
 def __init__(self, data):
    self.data_dict = data
    self.data = data.values()
    self.data_len = len(data)
  
 def __len__(self):
    #  worker_info = torch.utils.data.get_worker_info()
    #  dataset = worker_info.dataset
     return self.data_len

 def __getitem__(self, i):
     return self.data_dict[i]

 def __iter__(self):
  for x in self.data:
    worker = torch.utils.data.get_worker_info()
    worker_id = worker.id if worker is not None else -1
    start = time.time()
    time.sleep(0.1)
    end = time.time()
    yield x, worker_id, start, end

iterable_dataset = MyIterableDataset(data_dict)
# iterable_dataset = MyIterableDataset(data_list)

In [ ]:
def worker_init_fn(_):
 worker_info = torch.utils.data.get_worker_info()

 dataset = worker_info.dataset
 worker_id = worker_info.id
 split_size = len(dataset.data) // worker_info.num_workers
 dataset.data = list(dataset.data)[worker_id * split_size:(worker_id + 1) * split_size]

loader = DataLoader(iterable_dataset, batch_size=4, num_workers=2, worker_init_fn=worker_init_fn)

## Preparing Data

Here comes the preprocessing

In [ ]:
# tokenizer_W = WordPunctTokenizer()

# def tokenize_ru_in(x, tokenizer=tokenizer_W):
#     return tokenizer.tokenize(x.lower())[::-1]

# def tokenize_ru_out(x, tokenizer=tokenizer_W):
#     return tokenizer.tokenize(x.lower())

In [ ]:
# SRC = Field(tokenize=tokenize_ru_in,
#             init_token = '<sos>', 
#             eos_token = '<eos>', 
#             lower = True)

# TRG = Field(tokenize=tokenize_ru_out,
#             init_token = '<sos>', 
#             eos_token = '<eos>', 
#             lower = True)


# # dataset = torchtext.legacy.data.TabularDataset(
# #     path='data.txt',
# #     format='tsv',
# #     fields=[('trg', TRG), ('src', SRC)]
# # )

# dataset = torchtext.legacy.data.TabularDataset(
#     path="./data_for_seq2seq.csv",
#     format='csv',
#     skip_header=True,
#     fields=[('trg', TRG), ('src', SRC)],
#     # csv_reader_params = {'delimiter': '\t'}
# )

In [ ]:
print(len(iterable_dataset))
print(iterable_dataset[0][0]) #заголовок
print(iterable_dataset[0][1]) #аннотация
# print(iterable_dataset[0][2]) #статья

1021
The Statistical Mechanics of Black Hole Thermodynamics
['although', 'we', 'have', 'convincing', 'evidence', 'that', 'a', 'black', 'hole', 'bears', 'an', 'entropyproportional', 'to', 'its', 'surface', '(', 'horizon', ')', 'area', ',', 'the', '``', 'statistical', 'mechanical', "''", 'explanation', 'of', 'this', 'entropy', 'remains', 'unknown', '.', 'two', 'basic', 'questions', 'in', 'thisconnection', 'are', ':', 'what', 'is', 'the', 'microscopic', 'origin', 'of', 'the', 'entropy', ',', 'and', 'why', 'does', 'thelaw', 'of', 'entropy', 'increase', 'continue', 'to', 'hold', 'when', 'the', 'horizon', 'entropy', 'is', 'included', '?', 'after', 'a', 'review', 'of', 'some', 'of', 'the', 'difficulties', 'in', 'answering', 'these', 'questions', ',', 'ipropose', 'an', 'explanation', 'of', 'the', 'law', 'of', 'entropy', 'increase', 'which', 'comes', 'near', 'to', 'aproof', 'in', 'the', 'context', 'of', 'the', '``', 'semi', '-', 'classical', "''", 'approximation', ',', 'and', 'which', 'alsoprov

In [ ]:
# print(len(dataset.examples))
# print(dataset.examples[0].src) #статья наоборот
# print(dataset.examples[0].trg) #аннотация
# print(dataset.examples[0].leng)

In [ ]:
# train_data, valid_data, test_data = torch.utils.data.random_split(iterable_dataset, [0.8, 0.15, 0.05])

train_size = int(0.8 * len(iterable_dataset))
valid_size = int(0.15 * len(iterable_dataset))
test_size = len(iterable_dataset) - train_size - valid_size
train_data, valid_data, test_data = torch.utils.data.random_split(iterable_dataset, [train_size, valid_size, test_size])

print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

# train_data, valid_data, test_data = dataset.split(split_ratio=[0.8, 0.15, 0.05])

# print(f"Number of training examples: {len(train_data.examples)}")
# print(f"Number of validation examples: {len(valid_data.examples)}")
# print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 816
Number of validation examples: 153
Number of testing examples: 52


In [ ]:
# SRC.build_vocab(train_data, min_freq = 2)
# TRG.build_vocab(train_data, min_freq = 2)

In [ ]:
# print(f"Unique tokens in source (article) vocabulary: {len(SRC.vocab)}")
# print(f"Unique tokens in target (abstruct) vocabulary: {len(TRG.vocab)}")

And here is example from train dataset:

In [ ]:
print(train_data[8])

('Quantum Theory of Geometry II: Volume operators', ['a', 'functional', 'calculus', 'on', 'the', 'space', 'of', '(', 'generalized', ')', 'connections', 'was', 'recentlyintroduced', 'without', 'any', 'reference', 'to', 'a', 'background', 'metric', '.', 'it', 'is', 'used', 'to', 'continuethe', 'exploration', 'of', 'the', 'quantum', 'riemannian', 'geometry', '.', 'operators', 'corresponding', 'tovolume', 'of', 'three', '-', 'dimensional', 'regions', 'are', 'regularized', 'rigorously', '.', 'it', 'is', 'shownthat', 'there', 'are', 'two', 'natural', 'regularization', 'schemes', ',', 'each', 'of', 'which', 'leads', 'to', 'awell', '-', 'defined', 'operator', '.', 'both', 'operators', 'can', 'be', 'completely', 'specified', 'by', 'givingtheir', 'action', 'on', 'states', 'labelled', 'by', 'graphs', '.', 'the', 'two', 'final', 'results', 'are', 'closelyrelated', 'but', 'differ', 'from', 'one', 'another', 'in', 'that', 'one', 'of', 'the', 'operators', 'is', 'sensitiveto', 'the', 'differential', '

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
train_iterator = DataLoader(train_data, batch_size=4, num_workers=2, worker_init_fn=worker_init_fn)
valid_iterator = DataLoader(valid_data, batch_size=4, num_workers=2, worker_init_fn=worker_init_fn)
test_iterator = DataLoader(test_data, batch_size=4, num_workers=2, worker_init_fn=worker_init_fn)

In [ ]:
for i in  train_iterator:
  print(i)

AttributeError: ignored

In [ ]:
# def _len_sort_key(x):
#     return len(x.src)

# BATCH_SIZE = 128

# train_iterator, valid_iterator = BucketIterator.splits(
#     (train_data, valid_data), 
#     batch_size = BATCH_SIZE, 
#     device = device,
#     sort_key=_len_sort_key
# )

## Encoder

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout, bidirectional):
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.dropout = dropout
        self.bidirectional = bidirectional
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, num_layers=n_layers, dropout=dropout, bidirectional=bidirectional)
        
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, src):
        
        #src = [src sent len, batch size]
        
        # Compute an embedding from the src data and apply dropout to it
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src sent len, batch size, emb dim]
        
        # Compute the RNN output values of the encoder RNN. 
        # outputs, hidden and cell should be initialized here. Refer to nn.LSTM docs ;)
        
        _, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src sent len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        if self.bidirectional:
            #print(hidden.shape)
            hidden = hidden.reshape(self.n_layers, 2, -1, self.hid_dim)
            hidden = hidden.transpose(1, 2).reshape(self.n_layers, -1, 2 * self.hid_dim)

            cell = cell.reshape(self.n_layers, 2, -1, self.hid_dim)
            cell = cell.transpose(1, 2).reshape(self.n_layers, -1, 2 * self.hid_dim)
            #print(hidden.shape)
        return hidden, cell

## Decoder


In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.output_dim = output_dim
        self.n_layers = n_layers
        self.dropout = dropout
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, num_layers=n_layers, dropout=dropout)
        
        self.out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        # Compute an embedding from the input data and apply dropout to it
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        # Compute the RNN output values of the encoder RNN. 
        # outputs, hidden and cell should be initialized here. Refer to nn.LSTM docs ;)
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        
        #output = [sent len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #sent len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

## Seq2Seq

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        if encoder.bidirectional:
            assert encoder.hid_dim * 2 == decoder.hid_dim, \
                "Hidden dimensions of encoder and decoder must be equal!"
        else:
            assert encoder.hid_dim == decoder.hid_dim, \
                    "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            #get the highest predicted token from our predictions
            top1 = output.argmax(-1) 
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

## Training

In [ ]:
device

device(type='cuda')

In [ ]:
torch.cuda.is_available()

True

In [ ]:
y = torch.rand(5, 3)
y = y.cuda()

In [ ]:
# For reloading 
# import modules
# import imp
# imp.reload(modules)

# Encoder = modules.Encoder
# Attention = modules.Attention
# Decoder = modules.DecoderWithAttention
# Seq2Seq = modules.Seq2Seq

# INPUT_DIM = len(SRC.vocab)
# OUTPUT_DIM = len(TRG.vocab)
INPUT_DIM = 3000
OUTPUT_DIM = 3000

ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
BIDIRECTIONAL = False

# enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM // 2, N_LAYERS, ENC_DROPOUT, BIDIRECTIONAL)

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT, BIDIRECTIONAL)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

# dont forget to put the model to the right device
model = Seq2Seq(enc, dec, device)

In [ ]:
model = model.to(device)

RuntimeError: ignored

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(3000, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(3000, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (out): Linear(in_features=512, out_features=3000, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 10,431,416 trainable parameters


In [ ]:
# PAD_IDX = TRG.vocab.stoi['<pad>']
PAD_IDX = 1
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

def train(model, iterator, optimizer, criterion, clip, train_history=None, valid_history=None):
    model.train()
    
    epoch_loss = 0
    history = []
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg sent len, batch size]
        #output = [trg sent len, batch size, output dim]
        
        output = output[1:].view(-1, OUTPUT_DIM)
        trg = trg[1:].view(-1)
        
        #trg = [(trg sent len - 1) * batch size]
        #output = [(trg sent len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        # Let's clip the gradient
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
        history.append(loss.cpu().data.numpy())
        if (i+1)%10==0:
            fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 8))

            clear_output(True)
            ax[0].plot(history, label='train loss')
            ax[0].set_xlabel('Batch')
            ax[0].set_title('Train loss')
            if train_history is not None:
                ax[1].plot(train_history, label='general train history')
                ax[1].set_xlabel('Epoch')
            if valid_history is not None:
                ax[1].plot(valid_history, label='general valid history')
            plt.legend()
            
            plt.show()

        
    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    history = []
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg sent len, batch size]
            #output = [trg sent len, batch size, output dim]

            output = output[1:].view(-1, OUTPUT_DIM)
            trg = trg[1:].view(-1)

            #trg = [(trg sent len - 1) * batch size]
            #output = [(trg sent len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
import matplotlib
matplotlib.rcParams.update({'figure.figsize': (16, 12), 'font.size': 14})
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

In [ ]:
train_history = []
valid_history = []

N_EPOCHS = 12
CLIP = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP, train_history, valid_history)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'best-val-model.pt')
    
    train_history.append(train_loss)
    valid_history.append(valid_loss)
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 1m 21s
	Train Loss: 4.626 | Train PPL: 102.094
	 Val. Loss: 4.247 |  Val. PPL:  69.866
Epoch: 02 | Time: 1m 24s
	Train Loss: 4.353 | Train PPL:  77.693
	 Val. Loss: 3.524 |  Val. PPL:  33.930
Epoch: 03 | Time: 1m 25s
	Train Loss: 3.823 | Train PPL:  45.751
	 Val. Loss: 2.995 |  Val. PPL:  19.995
Epoch: 04 | Time: 1m 28s
	Train Loss: 3.634 | Train PPL:  37.849
	 Val. Loss: 2.748 |  Val. PPL:  15.616
Epoch: 05 | Time: 1m 39s
	Train Loss: 3.307 | Train PPL:  27.313
	 Val. Loss: 2.790 |  Val. PPL:  16.288
Epoch: 06 | Time: 2m 1s
	Train Loss: 3.285 | Train PPL:  26.696
	 Val. Loss: 2.762 |  Val. PPL:  15.835
Epoch: 07 | Time: 2m 40s
	Train Loss: 3.257 | Train PPL:  25.983
	 Val. Loss: 2.660 |  Val. PPL:  14.295
Epoch: 08 | Time: 3m 21s
	Train Loss: 3.194 | Train PPL:  24.378
	 Val. Loss: 2.578 |  Val. PPL:  13.169
Epoch: 09 | Time: 4m 34s
	Train Loss: 3.157 | Train PPL:  23.507
	 Val. Loss: 2.553 |  Val. PPL:  12.843
Epoch: 10 | Time: 5m 59s
	Train Loss: 3.159 | Train PPL:

Let's take a look at our network quality:

In [ ]:
def cut_on_eos(tokens_iter):
    for token in tokens_iter:
        if token == '<eos>':
            break
        yield token

def remove_tech_tokens(tokens_iter, tokens_to_remove=['<sos>', '<unk>', '<pad>']):
    return [x for x in tokens_iter if x not in tokens_to_remove]

def generate_translation(src, trg, model, TRG_vocab):
    model.eval()

    output = model(src, trg, 0) #turn off teacher forcing
    # output = output[1:].argmax(-1)

    original = remove_tech_tokens(cut_on_eos([TRG_vocab.itos[x] for x in list(trg[:,0].cpu().numpy())]))
    generated = remove_tech_tokens(cut_on_eos([TRG_vocab.itos[x] for x in list(output[:, 0].cpu().numpy())]))
    
    print('Original: {}'.format(' '.join(original)))
    print('Generated: {}'.format(' '.join(generated)))
    print()

def get_text(x, TRG_vocab):
     generated = remove_tech_tokens(cut_on_eos([TRG_vocab.itos[elem] for elem in list(x)]))
     return generated

In [ ]:
model.load_state_dict(torch.load('best-val-model.pt'))
batch = next(iter(valid_iterator))

for idx in range(10):
    src = batch.src[:, idx:idx+1]
    trg = batch.trg[:, idx:idx+1]
    generate_translation(src, trg, model, TRG.vocab)

Original: исследуется проблема научного дискурса с интегрального подхода . рассматриваются вопросы определения интегрального подхода в , дискурса , его и особое внимание уделяется . предлагается интегральная модель научного дискурса с его , , , и . намечаются перспективы дальнейшего исследования .
Generated: 

Original: ( больных . в , при . , на .
Generated: 



RuntimeError: ignored

# Text Summarization with Seq2Seq Model

## Step 1: Importing the Dataset

In [ ]:
import numpy as np # linear algebra
import pandas as pd


In [ ]:
# summary = pd.read_csv('news_summary.csv', encoding='iso-8859-1')
# raw = pd.read_csv('news_summary_more.csv', encoding='iso-8859-1')

In [ ]:
filename = "outoutGeneralRelativityAndQuantumCosmology_0_result.feather"
summary = pd.read_feather(filename)

In [ ]:
# pre1 =  raw.iloc[:,0:2].copy()
# pre1['head + text'] = pre1['headlines'].str.cat(pre1['text'], sep =" ") 

pre2 = summary.iloc[:,0:6].copy()
# pre2['text'] = pre2['author'].str.cat(pre2['date'].str.cat(pre2['read_more'].str.cat(pre2['text'].str.cat(pre2['ctext'], sep = " "), sep =" "),sep= " "), sep = " ")

In [ ]:
pre = pd.DataFrame()
pre['text'] = pre2['clear']
pre['summary'] = pre2['Annotation']

In [ ]:
pre.head()

,text,summary
0,\n\n \let\miguu=\footnote\n \def\footnote#1#2{...,Although we have convincing evidence that a b...
1,\nIn a modern Quantum Mechanics (QM) the parti...,We argue that correct account of the quantum ...
2,"\documentclass[12pt,a4paper]{article}\n\usepac...",We give a brief overview of black-hole soluti...
3,\nThe inflationary universe scenarios explain ...,Taking into account the effect of self-intera...
4,\font\mybb=msbm10 at 12pt\n\def\bbxx#1{\hbox{\...,It is argued that D=10 type II strings and M-...


In [ ]:
pre['text'][:10]

0    \n\n \let\miguu=\footnote\n \def\footnote#1#2{...
1    \nIn a modern Quantum Mechanics (QM) the parti...
2    \documentclass[12pt,a4paper]{article}\n\usepac...
3    \nThe inflationary universe scenarios explain ...
4    \font\mybb=msbm10 at 12pt\n\def\bbxx#1{\hbox{\...
5    \input harvmac\n\n\def\eg{ e.g.,\ }\n\def\ie{ ...
6    \nIf one attempts to quantise fields on acausa...
7    \n\n PACS numbers :04.20.Cv, 04.20.Fy, 04.20.M...
8    \nA fair amount of intuition about the kinemat...
9    \nBlack holes in 2-D space-times are toy model...
Name: text, dtype: object

## Step 2: Cleaning the Data

In [ ]:
import re

#Removes non-alphabetic characters:
def text_strip(column):
    for row in column:
        
        # #ORDER OF REGEX IS VERY VERY IMPORTANT!!!!!!
        
        # row=re.sub("(\\t)", ' ', str(row)).lower() #remove escape charecters
        # row=re.sub("(\\r)", ' ', str(row)).lower() 
        # row=re.sub("(\\n)", ' ', str(row)).lower()
        
        # row=re.sub("(__+)", ' ', str(row)).lower()   #remove _ if it occors more than one time consecutively
        # row=re.sub("(--+)", ' ', str(row)).lower()   #remove - if it occors more than one time consecutively
        # row=re.sub("(~~+)", ' ', str(row)).lower()   #remove ~ if it occors more than one time consecutively
        # row=re.sub("(\+\++)", ' ', str(row)).lower()   #remove + if it occors more than one time consecutively
        # row=re.sub("(\.\.+)", ' ', str(row)).lower()   #remove . if it occors more than one time consecutively
        
        # row=re.sub(r"[<>()|&©ø\[\]\'\",;?~*!]", ' ', str(row)).lower() #remove <>()|&©ø"',;?~*!
        
        # row=re.sub("(mailto:)", ' ', str(row)).lower() #remove mailto:
        # row=re.sub(r"(\\x9\d)", ' ', str(row)).lower() #remove \x9* in text
        # row=re.sub("([iI][nN][cC]\d+)", 'INC_NUM', str(row)).lower() #replace INC nums to INC_NUM
        # row=re.sub("([cC][mM]\d+)|([cC][hH][gG]\d+)", 'CM_NUM', str(row)).lower() #replace CM# and CHG# to CM_NUM
        
        
        # row=re.sub("(\.\s+)", ' ', str(row)).lower() #remove full stop at end of words(not between)
        # row=re.sub("(\-\s+)", ' ', str(row)).lower() #remove - at end of words(not between)
        # row=re.sub("(\:\s+)", ' ', str(row)).lower() #remove : at end of words(not between)
        
        # row=re.sub("(\s+.\s+)", ' ', str(row)).lower() #remove any single charecters hanging between 2 spaces
        
        # #Replace any url as such https://abc.xyz.net/browse/sdf-5327 ====> abc.xyz.net
        # try:
        #     url = re.search(r'((https*:\/*)([^\/\s]+))(.[^\s]+)', str(row))
        #     repl_url = url.group(3)
        #     row = re.sub(r'((https*:\/*)([^\/\s]+))(.[^\s]+)',repl_url, str(row))
        # except:
        #     pass #there might be emails with no url in them
        

        
        # row = re.sub("(\s+)",' ',str(row)).lower() #remove multiple spaces
        
        # #Should always be last
        # row=re.sub("(\s+.\s+)", ' ', str(row)).lower() #remove any single charecters hanging between 2 spaces

        
        
        yield row[::-1]

In [ ]:
brief_cleaning1 = text_strip(pre['text'])
brief_cleaning2 = text_strip(pre['summary'])

In [ ]:
from time import time
import spacy
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed
# nlp.max_length = 1700000 
#Taking advantage of spaCy .pipe() method to speed-up the cleaning process:
#If data loss seems to be happening(i.e len(text) = 50 instead of 75 etc etc) in this cell , decrease the batch_size parametre 

In [ ]:
%time
t = time()

#Batch the data points into 5000 and run on all cores for faster preprocessing
text = [str(doc) for doc in nlp.pipe(pre['text'][0], batch_size=5000,  n_process = -1)]

# text = brief_cleaning1[::-1]
# for x in brief_cleaning1:
#   break
# text = [str(doc) for doc in x]

#Takes 7-8 mins
print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

CPU times: user 22 µs, sys: 0 ns, total: 22 µs
Wall time: 37 µs
Time to clean up everything: 0.0 mins


In [ ]:
t = time()

#Batch the data points into 5000 and run on all cores for faster preprocessing

summary = ['_START_ '+ str(doc) + ' _END_' for doc in nlp.pipe(brief_cleaning2, batch_size=5000)]
#Takes 7-8 mins
print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 0.21 mins


In [ ]:
text[0]

IndexError: ignored

In [ ]:
summary[0]

'_START_  although we have convincing evidence that black hole bears an entropyproportional to its surface horizon area the ``statistical mechanical explanation of this entropy remains unknown two basic questions in thisconnection are what is the microscopic origin of the entropy and why does thelaw of entropy increase continue to hold when the horizon entropy is included after review of some of the difficulties in answering these questions ipropose an explanation of the law of entropy increase which comes near to aproof in the context of the ``semi-classical approximation and which alsoprovides proof in full quantum gravity under the assumption that the latterfulfills certain natural expectations like the existence of conserved energydefinable at infinity this explanation seems to require fundamentalspacetime discreteness in order for the entropy to be consistently finite andi recall briefly some of the ideas for what the discreteness might be if suchideas are right then our knowledge

## Step 3: Determining the Maximum Permissible Sequence Lengths

In [ ]:
# pre['cleaned_text'] = pd.Series(text)
pre['cleaned_text'] = pre['text']
pre['cleaned_summary'] = pd.Series(summary)

In [ ]:
text_count = []
summary_count = []

In [ ]:
for sent in pre['cleaned_text']:
    text_count.append(len(sent.split()))
for sent in pre['cleaned_summary']:
    summary_count.append(len(sent.split()))

AttributeError: ignored

In [ ]:
graph_df= pd.DataFrame()
graph_df['text']=text_count
graph_df['summary']=summary_count

In [ ]:
import matplotlib.pyplot as plt

graph_df.hist(bins = 5)
plt.show()

In [ ]:
#Check how much % of summary have 0-15 words
cnt=0
for i in pre['cleaned_summary']:
    if(len(i.split())<=15):
        cnt=cnt+1
print(cnt/len(pre['cleaned_summary']))

In [ ]:
#Check how much % of text have 0-70 words
cnt=0
for i in pre['cleaned_text']:
    if(len(i.split())<=100):
        cnt=cnt+1
print(cnt/len(pre['cleaned_text']))

In [ ]:
#Model to summarize the text between 0-15 words for Summary and 0-100 words for Text
max_text_len=100
max_summary_len=15

## Step 4: Selecting Plausible Texts and Summaries




In [ ]:
#Select the Summaries and Text between max len defined above

cleaned_text =np.array(pre['cleaned_text'])
cleaned_summary=np.array(pre['cleaned_summary'])

short_text=[]
short_summary=[]

for i in range(len(cleaned_text)):
    if(len(cleaned_summary[i].split())<=max_summary_len and len(cleaned_text[i].split())<=max_text_len):
        short_text.append(cleaned_text[i])
        short_summary.append(cleaned_summary[i])
        
post_pre=pd.DataFrame({'text':short_text,'summary':short_summary})

In [ ]:
post_pre.head(2)

In [ ]:
post_pre['summary'] = post_pre['summary'].apply(lambda x : 'sostok '+ x + ' eostok')

In [ ]:
post_pre.head(2)

## Step 5: Tokenizing the Text

In [ ]:
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(np.array(post_pre['text']),np.array(post_pre['summary']),test_size=0.1,random_state=0,shuffle=True)

In [ ]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences

#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer() 
x_tokenizer.fit_on_texts(list(x_tr))

In [ ]:
thresh=4

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in x_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

In [ ]:
#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences (i.e one-hot encodeing all the words)
x_tr_seq    =   x_tokenizer.texts_to_sequences(x_tr) 
x_val_seq   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr_seq,  maxlen=max_text_len, padding='post')
x_val   =   pad_sequences(x_val_seq, maxlen=max_text_len, padding='post')

#size of vocabulary ( +1 for padding token)
x_voc   =  x_tokenizer.num_words + 1

print("Size of vocabulary in X = {}".format(x_voc))

In [ ]:
#prepare a tokenizer for reviews on training data
y_tokenizer = Tokenizer()   
y_tokenizer.fit_on_texts(list(y_tr))

In [ ]:
thresh=6

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in y_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

In [ ]:
#prepare a tokenizer for reviews on training data
y_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
y_tokenizer.fit_on_texts(list(y_tr))

#convert text sequences into integer sequences (i.e one hot encode the text in Y)
y_tr_seq    =   y_tokenizer.texts_to_sequences(y_tr) 
y_val_seq   =   y_tokenizer.texts_to_sequences(y_val) 

#padding zero upto maximum length
y_tr    =   pad_sequences(y_tr_seq, maxlen=max_summary_len, padding='post')
y_val   =   pad_sequences(y_val_seq, maxlen=max_summary_len, padding='post')

#size of vocabulary
y_voc  =   y_tokenizer.num_words +1
print("Size of vocabulary in Y = {}".format(y_voc))

## Step 6: Removing Empty Texts and Summaries

In [ ]:
ind=[]
for i in range(len(y_tr)):
    cnt=0
    for j in y_tr[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_tr=np.delete(y_tr,ind, axis=0)
x_tr=np.delete(x_tr,ind, axis=0)

In [ ]:
ind=[]
for i in range(len(y_val)):
    cnt=0
    for j in y_val[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_val=np.delete(y_val,ind, axis=0)
x_val=np.delete(x_val,ind, axis=0)

## Step 7: Creating the Model

In [ ]:
from keras import backend as K 
import gensim
from numpy import *
import numpy as np
import pandas as pd 
import re
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

print("Size of vocabulary from the w2v model = {}".format(x_voc))

K.clear_session()

latent_dim = 300
embedding_dim=200

# Encoder
encoder_inputs = Input(shape=(max_text_len,))

#embedding layer
enc_emb =  Embedding(x_voc, embedding_dim,trainable=True)(encoder_inputs)

#encoder lstm 1
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

#encoder lstm 2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

#encoder lstm 3
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

#embedding layer
dec_emb_layer = Embedding(y_voc, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

#dense layer
decoder_dense =  TimeDistributed(Dense(y_voc, activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()

## Step 8: Training the Model

In [ ]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)

In [ ]:
history = model.fit(
    [x_tr, y_tr[:, :-1]],
    y_tr.reshape(y_tr.shape[0], y_tr.shape[1], 1)[:, 1:],
    epochs=50,
    callbacks=[es],
    batch_size=128,
    validation_data=([x_val, y_val[:, :-1]],
                     y_val.reshape(y_val.shape[0], y_val.shape[1], 1)[:
                     , 1:]),
    )

In [ ]:
from matplotlib import pyplot

pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=x_tokenizer.index_word
target_word_index=y_tokenizer.word_index

In [ ]:
# Inference Models

# Encode the input sequence to get the feature vector
encoder_model = Model(inputs=encoder_inputs, outputs=[encoder_outputs,
                      state_h, state_c])

# Decoder setup

# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim, ))
decoder_state_input_c = Input(shape=(latent_dim, ))
decoder_hidden_state_input = Input(shape=(max_text_len, latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2 = dec_emb_layer(decoder_inputs)

# To predict the next word in the sequence, set the initial states to the states from the previous time step
(decoder_outputs2, state_h2, state_c2) = decoder_lstm(dec_emb2,
        initial_state=[decoder_state_input_h, decoder_state_input_c])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2)

# Final decoder model
decoder_model = Model([decoder_inputs] + [decoder_hidden_state_input,
                      decoder_state_input_h, decoder_state_input_c],
                      [decoder_outputs2] + [state_h2, state_c2])

In [ ]:
def decode_sequence(input_seq):

    # Encode the input as state vectors.
    (e_out, e_h, e_c) = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1
    target_seq = np.zeros((1, 1))

    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        (output_tokens, h, c) = decoder_model.predict([target_seq]
                + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]

        if sampled_token != 'eostok':
            decoded_sentence += ' ' + sampled_token

        # Exit condition: either hit max length or find the stop word.
        if sampled_token == 'eostok' or len(decoded_sentence.split()) \
            >= max_summary_len - 1:
            stop_condition = True

        # Update the target sequence (of length 1)
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        (e_h, e_c) = (h, c)

    return decoded_sentence

In [ ]:
# To convert sequence to summary
def seq2summary(input_seq):
    newString = ''
    for i in input_seq:
        if i != 0 and i != target_word_index['sostok'] and i \
            != target_word_index['eostok']:
            newString = newString + reverse_target_word_index[i] + ' '

    return newString


# To convert sequence to text
def seq2text(input_seq):
    newString = ''
    for i in input_seq:
        if i != 0:
            newString = newString + reverse_source_word_index[i] + ' '

    return newString

In [ ]:
for i in range(0, 19):
    print ('Review:', seq2text(x_tr[i]))
    print ('Original summary:', seq2summary(y_tr[i]))
    print ('Predicted summary:', decode_sequence(x_tr[i].reshape(1,
           max_text_len)))
    print '\n'